<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ContourExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.5 MB/s eta 0:00:00


In [2]:
import numpy as np
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.md.contour_exploration import ContourExploration
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
bulk_Al_settings = {
    'maxstep': 1.0,
    'parallel_drift': 0.05,
    'remove_translation': True,
    'potentiostat_step_scale': None,
    'use_frenet_serret': True,
    'angle_limit': 20,
    'loginterval': 1}
size = 2
atoms = bulk('Al', 'fcc', cubic=True).repeat((size, size, size))
atoms.calc = EMT()

name = 'test_potentiostat'
seed = 19460926


E0 = atoms.get_potential_energy()

atoms.rattle(stdev=0.18, seed=seed)
initial_energy = atoms.get_potential_energy()

rng = np.random.RandomState(seed)
MaxwellBoltzmannDistribution(atoms, temperature_K=300, rng=rng)
with ContourExploration(
    atoms,
    **bulk_Al_settings,
    energy_target=initial_energy,
    rng=rng,
    trajectory=name + '.traj',
    logfile=name + '.log',
) as dyn:
    print("Energy Above Ground State: {: .4f} eV/atom".format(
        (initial_energy - E0) / len(atoms)))
    for _ in range(5):
        dyn.run(5)
        energy_error = (atoms.get_potential_energy() -
                        initial_energy) / len(atoms)
        print(f'Potentiostat Error {energy_error: .4f} eV/atom')
        # assert 0 == pytest.approx(energy_error, abs=0.01)

Energy Above Ground State:  0.1764 eV/atom
Potentiostat Error -0.0027 eV/atom
Potentiostat Error -0.0008 eV/atom
Potentiostat Error -0.0029 eV/atom
Potentiostat Error -0.0006 eV/atom
Potentiostat Error -0.0008 eV/atom


In [3]:

from ase.lattice.cubic import FaceCenteredCubic
#from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.optimize import BFGS
import os
#from contour_exploration import contour_exploration
from ase.md.contour_exploration import ContourExploration
from ase import units, io, Atom

import numpy as np

# Use Asap for a huge performance increase if it is installed
use_asap = False

if use_asap:
    from asap3 import EMT
else:
    from ase.calculators.emt import EMT


#pds       = [0.0, 0.05, 0.1, 0.2]
pds        = [0.2] # only value used in the paper
#mxs       = [0.05, 0.1, 0.2, 0.4]
mxs        = [0.2] # only value used in the paper
#anglelims = [5, 10 ,20]
anglelims  = [5, 20] # only values used  in the paper
energy_barriers_scales = [0.5, 1.0, 1.5]


seed = 453 #'ASE'
rng = np.random.default_rng(seed)

for parallel_drift in pds:
    for maxstep in mxs:
        for angle_limit in anglelims:
            for ebx in energy_barriers_scales:

                traj_name = 'atom_on_frozen_surface_test_%.2f_pd_%.2f_mxstp_%02d_angle_%.2f_ebx.traj' % (parallel_drift, maxstep, angle_limit,ebx)

                fmax_relax = 1e-6
                size_xy = 1
                size_z = 1

                atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                                          symbol='Al',
                                          size=(size_xy, size_xy, size_z),
                                          pbc=True)

                if False:
                    shift=  np.dot(np.array([-0.25, -0.25, 0.0])/size_xy,atoms.cell)
                    atoms.translate(shift)
                    atoms.wrap()


                from ase.constraints import FixAtoms
                c = FixAtoms(indices=[atom.index for atom in atoms])
                atoms.set_constraint(c)

                atoms.append(
                    Atom('Cu',
                        #position = (0.1, 0.1, atoms.cell[2,2] - .3) ))
                        position = (atoms.cell[0,0]/2., atoms.cell[1,1]/2., atoms.cell[2,2] - .3) ))

                atoms.cell[2,2] = 4* atoms.cell[2,2]

                atoms.set_calculator(EMT())

                ############### ground state

                relax = BFGS(atoms,trajectory='surface_relax.traj')
                relax.run(fmax=fmax_relax)
                E0 = atoms.get_potential_energy()
                io.write('initial.traj',atoms)
                atoms0 = atoms.copy()


                ########### barrier with dimer
                from ase.dimer import DimerControl, MinModeAtoms, MinModeTranslate, normalize

                #### Set up the dimer
                fdstep = 0.5
                displacement_vector = [[0.0]*3]*(len(atoms))
                displacement_vector[-1] = [0.1, 0.1, 0.1]
                displacement_vector = fdstep * normalize(displacement_vector)


                #
                d_control = DimerControl(initial_eigenmode_method = 'displacement',
                                        dimer_separation = 0.00001,
                                        #extrapolate_forces = False,
                                        #use_central_forces = False,
                                        displacement_method = 'vector', logfile = 'dimer.log' )
                                        #mask = dimer_mask) # we don't need a mask with all other atoms frozen
                d_atoms = MinModeAtoms(atoms, d_control)

                # Displace the atoms
                d_atoms.displace(displacement_vector =  displacement_vector)

                # Converge to a saddle point
                dim_rlx = MinModeTranslate(d_atoms, trajectory = 'dimer_method.traj', )
                                           #logfile = logfile)
                dim_rlx.run(fmax = fmax_relax)

                ### post processing
                #print('DimerControl\n',dir(d_control))
                #print('MinModeAtoms\n',dir(d_atoms))
                #print('MinModeTranslate\n',dir(dim_rlx))

                eigenmode =  d_atoms.get_eigenmode()
                print('dimer_eigenmode:\n', d_atoms.get_eigenmode())
                print('dimer_curvature:\n', d_atoms.get_curvature())
                #atoms.set_positions( atoms.get_positions()+ 0.5*dimer_direction) # gets the middle of the dimer

                saddle = atoms.copy()
                saddle.calc = EMT()

                Eb = saddle.get_potential_energy()
                forcesb = saddle.get_forces()
                io.write('saddle.traj',saddle)

                maxforceb = np.sqrt( (forcesb ** 2).sum(axis=1).max() )
                print("saddle forces:\n", forcesb)
                print("saddle forces max", maxforceb)


                barrier_energy = Eb-E0
                print('Saddle barrier',barrier_energy)

                ############ CED setup

                atoms.set_positions(atoms0.get_positions())
                # Set the momenta corresponding to T=300K
                #MaxwellBoltzmannDistribution(atoms, 300 * units.kB)

                #p = np.zeros((len(atoms),3)) #atoms.get_momenta()
                #p[-1] = [-1.0, -1.0, 0.0]
                atoms.set_momenta(eigenmode)
                #atoms.set_momenta( p -  np.sum(p,axis =0)/len(atoms))
                atoms.rattle(stdev=0.15 , seed = 60622)

                print('initial momenta',atoms.get_momenta())


                #extra_energy = 0.350 # just above the barrier, Barrier = 0.3196
                extra_energy = ebx* barrier_energy

                energy_target = E0 + extra_energy

                def printenergy(i,a):
                    """Function to print the potential, kinetic and total energy"""
                    epot = a.get_potential_energy()
                    dev = epot - energy_target

                    print(i,'Energy Target: {: 6f}, Epot {: 6f} eV,  Deviation from target {: 6f} meV'.format( energy_target, epot, dev*1000))


                #energy_target = 1.28


                logfile=  traj_name.replace('.traj', '.log')
                if os.path.isfile(logfile): os.remove(logfile)

                ## pre-release settings/parameters, renamed or
                ## slightly improved for final release
                # dyn = contour_exploration(atoms,
                #                 maxstep = maxstep,
                #                 parallel_drift = parallel_drift,
                #                 remove_translation  = False,
                #                 force_parallel_step_scale = None,
                #                 energy_target = energy_target,
                #                 initialize_old = True, use_FS = True,
                #                 use_target_shift = True,
                #                 angle_limit = angle_limit,
                #                 trajectory = traj_name,
                #                 logfile = logfile )

                dyn = ContourExploration(
                    atoms,
                    maxstep = maxstep,
                    parallel_drift = parallel_drift,
                    remove_translation  = False,
                    potentiostat_step_scale = None,
                    energy_target = energy_target,
                    use_frenet_serret = True,
                    use_target_shift = True,
                    target_shift_previous_steps=10,
                    angle_limit = angle_limit,
                    trajectory = traj_name,
                    logfile = logfile,
                    rng=rng)



                # Now run the dynamics
                printenergy(0,atoms)
                for i in range(10000):
                    dyn.run(1)
                    printenergy(i,atoms)




                if False:
                    ###########
                    from ase import io
                    traj = io.Trajectory(traj_name, 'r')

                    es = equilibration_steps = 20

                    energy = np.array( [im.get_potential_energy() for im in traj] )
                    mean_deviation = np.mean(energy[es:]-energy_target)#/len(atoms)
                    rms_deviation = np.sqrt(np.mean((energy[es:]-energy_target)**2))#/len(atoms)
                    standard_deviation = np.std(energy[es:])#/len(atoms)
                    print("Images",len(traj))
                    print("Mean deviation %f" %mean_deviation)
                    print("Standard deviation %f"% standard_deviation)
                    print("RMS deviation %f"% rms_deviation)





                    X = []
                    Y = []

                    for image in traj:
                        image.wrap()
                        pos = image.get_positions()
                        X.append(pos[-1,0] )
                        Y.append(pos[-1,1])

                    #traj.close()



                    import matplotlib.pyplot as plt
                    from ase.visualize.plot import plot_atoms

                if False:
                    fig, ax = plt.subplots(figsize = (3.2,2.5), dpi = 150)

                    offset = .3085
                    plot_atoms(atoms0, ax, offset=(-offset, -offset))

                    ax.set_xlim(-0.4, 4.4)
                    ax.set_ylim(-0.4, 4.4)
                    plt.plot(X,Y, marker = 'o', linestyle = '', markersize = 2, color = 'b')

                    fig.tight_layout(pad= 0.3)

                    fig.savefig(traj_name.replace('.traj', '.pdf'), transparent = True)


                if False:

                    import matplotlib.pyplot as plt

                    fig, ax = plt.subplots(figsize = (3.2,2.5), dpi = 150)
                    ax.axhline(0, color = 'grey', linewidth = 0.5, linestyle = '-')
                    ax.axhline(per_atom_energy*1000, color = 'grey', linewidth = 0.5, linestyle = '-')
                    ax.plot((energy-E0) *1000, color = 'teal')

                    ax.set_ylabel('$E - E_{contour}$ (meV)')
                    ax.set_xlabel('Iteration #')
                    ax.minorticks_on()

                    fig.tight_layout(pad= 0.3)

                    fig.savefig(traj_name.replace('.traj', '_energy.pdf'), transparent = True)




                if False:

                    # kwargs can be any ase.io.utils.PlottingVariables.__init__() input

                    from ase.io.animation import write_gif
                    from ase import io
                    write_gif(traj_name.replace('.traj', '.gif'), traj,
                            interval = 200,  show_unit_cell = 2, rotation = '0x, 0y, 0z', )



                if False: plt.show()


      Step     Time          Energy          fmax
BFGS:    0 18:29:27        2.182145        0.037333
BFGS:    1 18:29:27        2.182125        0.035584
BFGS:    2 18:29:27        2.181933        0.000242
BFGS:    3 18:29:27        2.181933        0.000002
BFGS:    4 18:29:27        2.181933        0.000000


<ipython-input-3-739de211c0f0>:65: DeprecationWarning: Please use atoms.calc = calc
  atoms.set_calculator(EMT())


ImportError: cannot import name 'normalize' from 'ase.dimer' (/usr/local/lib/python3.10/dist-packages/ase/dimer.py)